In [32]:
pip install sentence-transformers


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 95.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 73.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 45.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 777.0 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 11.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 70.2 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitli

In [35]:
!pip install pdfplumber sentence-transformers transformers langdetect

In [36]:
# ============================
# 1. Gerekli Kütüphaneler
# ============================
# !pip install pdfplumber sentence-transformers transformers langdetect

import pdfplumber
from langdetect import detect
from sentence_transformers import SentenceTransformer, util
from transformers import pipeline

# ============================
# 2. Modelleri Yükleme
# ============================
# Semantik benzerlik modeli
semantic_model = SentenceTransformer('all-MiniLM-L6-v2')

# Zero-shot sınıflandırma modeli (bölüm tespiti için)
classifier = pipeline("zero-shot-classification", model="facebook/bart-large-mnli")

# ============================
# 3. Kriterler ve Ağırlıklar
# ============================
criteria_descriptions = {
    "iş_deneyimi": "Work experience, previous roles, responsibilities, achievements, and career growth.",
    "egitim": "Education background, degrees, GPA, and academic achievements.",
    "teknik_beceriler": "Technical skills such as programming, software knowledge, and certifications.",
    "ozet": "A summary or personal statement highlighting career goals and strengths.",
    "liderlik": "Leadership and organizational experiences in clubs, projects, or initiatives.",
    "sertifikalar": "Certifications, courses, and training programs completed.",
    "iletisim": "Contact information including phone, email, LinkedIn.",
    "portfolyo": "Portfolio links, GitHub, personal websites.",
    "diller": "Language skills and proficiency levels.",
    "referanslar": "References or referees with contact details."
}

criteria_weights = {
    "iş_deneyimi": 25,
    "egitim": 15,
    "teknik_beceriler": 15,
    "ozet": 10,
    "liderlik": 10,
    "sertifikalar": 10,
    "iletisim": 5,
    "portfolyo": 5,
    "diller": 3,
    "referanslar": 2
}

# ============================
# 4. PDF Metin Çıkarma
# ============================
def extract_text_from_pdf(file_path):
    text = ""
    with pdfplumber.open(file_path) as pdf:
        for page in pdf.pages:
            page_text = page.extract_text()
            if page_text:
                text += page_text + "\n"
    return text.strip()

# ============================
# 5. Zero-Shot Bölümleme
# ============================
def split_cv_sections(text):
    labels = list(criteria_descriptions.keys())
    paragraphs = [p.strip() for p in text.split("\n") if p.strip()]
    section_map = {label: "" for label in labels}

    for paragraph in paragraphs:
        result = classifier(paragraph, candidate_labels=labels)
        best_label = result["labels"][0]
        section_map[best_label] += paragraph + " "
    return section_map

# ============================
# 6. Semantik Benzerlik Skoru
# ============================
def calculate_similarity(section_text, ideal_text):
    if not section_text.strip():
        return 0.0
    emb_section = semantic_model.encode(section_text, convert_to_tensor=True)
    emb_ideal = semantic_model.encode(ideal_text, convert_to_tensor=True)
    score = util.cos_sim(emb_section, emb_ideal).item()
    return round(score, 2)

# ============================
# 7. CV Analiz Fonksiyonu
# ============================
def analyze_cv_semantic(text):
    try:
        lang = detect(text)
    except:
        lang = "unknown"

    sections = split_cv_sections(text)
    total_score = 0
    strengths = []
    weaknesses = []

    for section, description in criteria_descriptions.items():
        score = calculate_similarity(sections[section], description)
        weighted_score = score * criteria_weights[section]
        total_score += weighted_score

        if score > 0.6:
            strengths.append(f"{section.replace('_',' ').title()} bölümü iyi (benzerlik: {score}).")
        elif 0.3 < score <= 0.6:
            weaknesses.append(f"{section.replace('_',' ').title()} bölümü zayıf (benzerlik: {score}).")
        else:
            weaknesses.append(f"{section.replace('_',' ').title()} bölümü eksik.")

    return {
        "language": lang,
        "total_score": round(total_score, 2),
        "strengths": strengths,
        "weaknesses": weaknesses
    }

# ============================
# 8. Manuel PDF Yükleme (Colab için)
# ============================
def manual_pdf_upload():
    try:
        from google.colab import files
        print("Lütfen analiz etmek istediğiniz PDF dosyasını seçin:")
        uploaded = files.upload()
        pdf_path = list(uploaded.keys())[0]
        print(f"'{pdf_path}' başarıyla yüklendi.")
        return pdf_path
    except ImportError:
        return input("PDF dosyasının tam yolunu girin: ")

# ============================
# 9. Ana Program
# ============================
if __name__ == "__main__":
    pdf_path = manual_pdf_upload()
    cv_text = extract_text_from_pdf(pdf_path)
    report = analyze_cv_semantic(cv_text)

    print(f"\nCV Dili: {report['language']}")
    print(f"Toplam Puan: {report['total_score']}/100\n")

    print("Güçlü Yönler:")
    for s in report['strengths']:
        print(f" - {s}")

    print("\nZayıf Yönler:")
    for w in report['weaknesses']:
        print(f" - {w}")


config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

Device set to use cpu


Lütfen analiz etmek istediğiniz PDF dosyasını seçin:


Saving Curriculum Vitae - Meriç Özcan.pdf to Curriculum Vitae - Meriç Özcan (7).pdf
'Curriculum Vitae - Meriç Özcan (7).pdf' başarıyla yüklendi.

CV Dili: en
Toplam Puan: 16.18/100

Güçlü Yönler:

Zayıf Yönler:
 - Iş Deneyimi bölümü eksik.
 - Egitim bölümü zayıf (benzerlik: 0.34).
 - Teknik Beceriler bölümü eksik.
 - Ozet bölümü eksik.
 - Liderlik bölümü zayıf (benzerlik: 0.34).
 - Sertifikalar bölümü eksik.
 - Iletisim bölümü eksik.
 - Portfolyo bölümü zayıf (benzerlik: 0.55).
 - Diller bölümü eksik.
 - Referanslar bölümü eksik.
